In [ ]:
# Import core libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Import model components
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Conv2D, MaxPool2D, BatchNormalization, 
                                   Dropout, Dense, Flatten, GaussianNoise)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import (LearningRateScheduler, 
                                      ModelCheckpoint, EarlyStopping)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Optimizer with weight decay from TensorFlow Addons
from tensorflow_addons.optimizers import AdamW

c:\Users\ayush\miniconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\ayush\miniconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you wa

In [3]:
# Limit GPU memory available to TensorFlow

# Get list of GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20480)]
    )

In [ ]:
# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape data to (num_samples, height, width, channels) and normalize to [0,1]
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

# Convert class labels to one-hot encoded vectors
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Configure data augmentation pipeline
datagen = ImageDataGenerator(
    rotation_range=15,       # Random rotations ±15 degrees
    zoom_range=0.2,          # Random zoom up to 20%
    width_shift_range=0.15,  # Horizontal shift ±15% of width
    height_shift_range=0.15, # Vertical shift ±15% of height
    shear_range=0.1,         # Shear transformations
    validation_split=0.2,    # Hold out 20% for validation
    preprocessing_function=lambda x: x + np.random.normal(0, 0.05, x.shape) # Add Gaussian noise
)

In [ ]:
# Build CNN architecture
net = Sequential([
    # Input layer with Gaussian noise for regularization
    GaussianNoise(0.1, input_shape=(28,28,1)),
    
    # First convolutional block
    Conv2D(64, (3,3), activation='swish', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),  # Normalize activations before passing to next layer
    Conv2D(64, (3,3), activation='swish', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPool2D((2,2)),      # Downsample by factor of 2 in both dimensions
    Dropout(0.25),         # Randomly set 25% of activations to zero
    
    # Second convolutional block with more filters
    Conv2D(128, (3,3), activation='swish', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='swish', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPool2D((2,2)),
    Dropout(0.35),        # Increased dropout for deeper layers to prevent overfitting
    
    # Final convolutional block with even more filters
    Conv2D(256, (3,3), activation='swish', padding='valid', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Conv2D(256, (3,3), activation='swish', padding='valid', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.5),         # High dropout before dense layers to prevent overfitting
    
    # Classification head with dense layers
    Flatten(),            # Convert 3D features to 1D for dense layers
    Dense(1024, activation='swish', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='swish', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(10, activation='softmax')  # Output layer with class probabilities, 10 different classes (digits 0 through 9) with softmax activation (assigns class with highest probability)
])

In [ ]:
# Learning rate schedule function for training
def lr_schedule(epoch):
    lr = 3e-4   # Initial learning rate for AdamW optimizer
    if epoch > 30: lr *= 0.1    # Reduce LR by 10x after 30 epochs 
    if epoch > 45: lr *= 0.1    # Reduce again after 45 epochs 
    return lr

# Configure model for training
net.compile(optimizer=AdamW(weight_decay=1e-5),   # Adam with weight decay optimizer 
              loss='categorical_crossentropy',    # Suitable for multi-class classification 
              metrics=['accuracy'])               # Monitor accuracy during training 

In [ ]:
# Train the model with augmented data and callbacks for learning rate schedule, model checkpointing, and early stopping
history = net.fit(
    training_data=datagen.flow(x_train, y_train, batch_size=256, subset='training'), # Training data generator 
    validation_data=datagen.flow(x_train, y_train, subset='validation'),             # Validation generator 
    epochs=60,
    callbacks=[
        LearningRateScheduler(lr_schedule),                    # Dynamic learning rate schedule       
        ModelCheckpoint('model.h5', save_best_only=True),      # Save best weights to file
        EarlyStopping(patience=12, restore_best_weights=True)  # Stop if no improvement after 12 epochs 
    ]
)

Epoch 1/60
188/188 [==============================] - 19s 76ms/step - loss: 1.5960 - accuracy: 0.6064 - val_loss: 10.7308 - val_accuracy: 0.1126 - lr: 3.0000e-04
Epoch 2/60
188/188 [==============================] - 14s 76ms/step - loss: 0.6160 - accuracy: 0.8956 - val_loss: 2.5713 - val_accuracy: 0.3443 - lr: 3.0000e-04
Epoch 3/60
188/188 [==============================] - 16s 85ms/step - loss: 0.4918 - accuracy: 0.9336 - val_loss: 0.5301 - val_accuracy: 0.9183 - lr: 3.0000e-04
Epoch 4/60
188/188 [==============================] - 15s 82ms/step - loss: 0.4323 - accuracy: 0.9527 - val_loss: 0.4445 - val_accuracy: 0.9527 - lr: 3.0000e-04
Epoch 5/60
188/188 [==============================] - 16s 85ms/step - loss: 0.3987 - accuracy: 0.9611 - val_loss: 0.3598 - val_accuracy: 0.9758 - lr: 3.0000e-04
Epoch 6/60
188/188 [==============================] - 16s 83ms/step - loss: 0.3744 - accuracy: 0.9679 - val_loss: 0.3608 - val_accuracy: 0.9749 - lr: 3.0000e-04
Epoch 7/60
188/188 [=============

In [ ]:
# Load best performing version of the model, and check accuracy against test data (not used during training)
final_model = load_model('model.h5')
loss, acc = final_model.evaluate(x_test, y_test)
print(f"Test accuracy: {acc*100:.2f}%")

313/313 [==============================] - 2s 4ms/step - loss: 0.0900 - accuracy: 0.9964
Test accuracy: 99.64%


In [ ]:
# Save model in HDF5 format
final_model.save("model.h5")